# Starfinder: Soldier with Magnetar Rifle

Advanced weapons are available to all martial characters through the {srd_feats}`Weapon Proficiency <911-weapon-proficiency>` general feat.
However, with Weapon Proficiency, when you hit level 5 your proficiency with martial weapons increases to expert, but not that with advanced weapons. This is unlike ancestral Weapon Familiarity feats.

The {srd_weapons}`Magnetar Rifle <68-magnetar-rifle>` is an advanced weapon (d12 Analog, Automatic, range 60ft, magazine 30).
The closest match with martial weapons is the substantially worse {srd_weapons}`Rotolaser <58-rotolaser>` (d8 Automatic, Tech, range 30ft, magazine 10).

Crucially, Area Fire and Auto-Fire use one's class proficiency, not the weapon proficiency - which raises the question of how do the damage profiles for the two weapons compare.
So a {srd_classes}`Soldier<5-soldier>` using a Magnetar Rifle would use their lowered weapon proficiency for Primary Target and simple Strikes, and their full class proficiency for Auto-fire.

Let's analyse a full round of firing a rotolaser and compare it with a magnetar rifle.
With both weapons, we'll do Primary target -> Auto-Fire against a single target -> simple Strike.

In [ ]:
import xarray

import pathfinder2e_stats as pf2

%matplotlib inline

In [ ]:
level = 5

atk = xarray.DataArray(
    [14, 12],  # level 5 + trained 2 or expert 4 + dex 4 + tracking 1
    dims=["weapon"],
    coords={"weapon": ["rotolaser", "magnetar_rifle"]},
)

area_fire_DC = 22  # 10 + level 5 + trained 2 + con 4 + tracking 1

rotolaser = pf2.Damage("fire", 2, 8)
magnetar_rifle = pf2.Damage("piercing", 2, 12)

enemy = pf2.tables.SIMPLE_NPC.sel(level=level, drop=True)[["AC", "saving_throws", "HP"]]
enemy.to_pandas()

In [ ]:
# both 'weapon' and 'challenge' are what-if analyses - let's compare the same dice rolls
# against progressively harder-to-hit enemies.
pf2.set_config(
    check_dependent_dims=("challenge", "weapon"),
    damage_dependent_dims=("challenge",),
)

In [ ]:
primary_target = pf2.check(atk, DC=enemy.AC)
primary_target = xarray.concat(
    [
        pf2.damage(primary_target.sel(weapon="rotolaser"), rotolaser),
        pf2.damage(primary_target.sel(weapon="magnetar_rifle"), magnetar_rifle),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
auto_fire = pf2.check(
    enemy.saving_throws, DC=area_fire_DC, primary_target=primary_target
)

auto_fire = xarray.concat(
    [
        pf2.damage(
            auto_fire.sel(weapon="rotolaser"),
            rotolaser.copy(basic_save=True),
        ),
        pf2.damage(
            auto_fire.sel(weapon="magnetar_rifle"),
            magnetar_rifle.copy(basic_save=True),
        ),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
# Note: Primary Target does not increase MAP, but Auto-Fire does
third_strike = pf2.check(atk - 5, DC=enemy.AC)
third_strike = xarray.concat(
    [
        pf2.damage(third_strike.sel(weapon="rotolaser"), rotolaser),
        pf2.damage(third_strike.sel(weapon="magnetar_rifle"), magnetar_rifle),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
full_round = xarray.concat([primary_target, auto_fire, third_strike], dim="action")
full_round["action"] = ["primary_target", "auto_fire", "third_strike"]
full_round

## Chance to hit
Note how the saving throw against auto-fire uses the same DC for both weapons, but it is influenced by the outcome of the Primary Target strike.

In [ ]:
pf2.outcome_counts(full_round).stack(
    row=["action", "outcome"], col=["challenge", "weapon"]
).to_pandas()

## Mean damage

In [ ]:
total_damage = full_round.total_damage.mean("roll")
total_damage = xarray.concat(
    [total_damage, total_damage.sum("action").expand_dims(action=["TOTAL"])],
    dim="action",
)
total_damage = total_damage.stack(col=["challenge", "weapon"]).to_pandas()
total_damage

## Damage distribution

In [ ]:
bins = full_round.total_damage.max().item() + 1
_ = (
    full_round.total_damage.stack(col=["challenge", "weapon"])
    .sum("action")
    .to_pandas()
    .hist(bins=bins, sharex=True, figsize=(10, 10))
)